In [1]:
%matplotlib inline

from datetime import datetime, date
import pandas as pd
import yfinance as yf
from pykalman import KalmanFilter
import numpy as np
import random

import glob
import os
import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash import dash_table as dt
import dash_table.FormatTemplate as FormatTemplate

from dash.dash_table.Format import Group
import plotly.express as px

C:\Users\NLISIC~1\AppData\Local\Temp/ipykernel_3428/1950727162.py:18: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\NLISIC~1\AppData\Local\Temp/ipykernel_3428/1950727162.py:19: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\NLISIC~1\AppData\Local\Temp/ipykernel_3428/1950727162.py:23: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table.FormatTemplate as FormatTemplate


# Load Data

In [2]:
local_dir = 'input_data/saved_data_tables'

author_data = pd.read_csv(f'{local_dir}/author_data.csv', index_col=None, header=0)
comment_data = pd.read_csv(f'{local_dir}/comment_data.csv', index_col=None, header=0)
submission_data = pd.read_csv(f'{local_dir}/submission_data.csv', index_col=None, header=0)
text_data = pd.read_csv(f'{local_dir}/text_data.csv', index_col=None, header=0)
sentiment_data = pd.read_csv(f'{local_dir}/sentiment_data.csv', index_col=None, header=0)
ticker_data = pd.read_csv(f'{local_dir}/ticker_data.csv', index_col=None, header=0)
submission_comment_data = pd.read_csv(f'{local_dir}/submission_comment_data.csv', index_col=None, header=0)
flatten_data = pd.read_csv(f'{local_dir}/flatten_data.csv', index_col=None, header=0)

author_data['created_utc'] = pd.to_datetime(author_data['created_utc'], format='%Y-%m-%d %H:%M:%S')
author_data['update_dt'] = pd.to_datetime(author_data['update_dt'], format='%Y-%m-%d %H:%M:%S')

comment_data['created_utc'] = pd.to_datetime(comment_data['created_utc'], format='%Y-%m-%d %H:%M:%S')
comment_data['update_dt'] = pd.to_datetime(comment_data['update_dt'], format='%Y-%m-%d %H:%M:%S')

submission_data['created_utc'] = pd.to_datetime(submission_data['created_utc'], format='%Y-%m-%d %H:%M:%S')
submission_data['update_dt'] = pd.to_datetime(submission_data['update_dt'], format='%Y-%m-%d %H:%M:%S')

text_data['update_dt'] = pd.to_datetime(text_data['update_dt'], format='%Y-%m-%d %H:%M:%S')

submission_comment_data['created_utc'] = pd.to_datetime(submission_comment_data['created_utc'], format='%Y-%m-%d %H:%M:%S')
submission_comment_data['update_dt'] = pd.to_datetime(submission_comment_data['update_dt'], format='%Y-%m-%d %H:%M:%S')

flatten_data['created_utc'] = pd.to_datetime(flatten_data['created_utc'], format='%Y-%m-%d %H:%M:%S')
flatten_data['update_dt'] = pd.to_datetime(flatten_data['update_dt'], format='%Y-%m-%d %H:%M:%S')

In [3]:
flatten_data.head()

,id,type,negative,neutral,positive,compound,sentiment,ticker,author_id,score,created_utc,update_dt,comment_karma,link_karma,awarder_karma,awardee_karma,total_karma
0,qo4gnp,submission,0.184,0.747,0.069,-0.8718,negative,GOOG,845k3tg2,4.0,2021-11-06 17:01:17,2021-11-06 17:00:00,143.0,112.0,0.0,56.0,311.0
1,qo3u6q,submission,0.046,0.826,0.128,0.7227,positive,IRS,8spa6hs8,38.0,2021-11-06 16:29:41,2021-11-06 17:00:00,7862.0,9239.0,835.0,803.0,18739.0
2,qo3u6q,submission,0.046,0.826,0.128,0.7227,positive,RH,8spa6hs8,38.0,2021-11-06 16:29:41,2021-11-06 17:00:00,7862.0,9239.0,835.0,803.0,18739.0
3,hjkft58,comment,0.328,0.526,0.145,-0.6338,negative,GO,6mpxtvr5,9.0,2021-11-06 16:33:53,2021-11-06 17:00:00,8912.0,2608.0,94.0,1503.0,13117.0
4,qo6px6,submission,0.052,0.822,0.127,0.9804,positive,MLI,3jrwyqto,3.0,2021-11-06 18:54:19,2021-11-06 19:00:00,1915.0,28.0,0.0,53.0,1996.0


# Get Ticker Sentiment

In [4]:

import datetime as dt
def get_agg_ticker_sentiment_top_N(df_source, submission_type = 'all', score_min = 0, date_window = 'all', top_records = 10):
    filter_type = ['submission', 'comment']
    if submission_type == 'submission':
        filter_type = ['submission']
    elif submission_type == 'comment':
        filter_type = ['comment']
    
    filter_date = dt.datetime.now()-dt.timedelta(days=365)
    if date_window == '6h':
        filter_date = dt.datetime.now()-dt.timedelta(hours=6)
    elif date_window == '12h':
        filter_date = dt.datetime.now()-dt.timedelta(hours=12)
    elif date_window == '24h':
        filter_date = dt.datetime.now()-dt.timedelta(hours=24)
    elif date_window == '48h':
        filter_date = dt.datetime.now()-dt.timedelta(hours=48)
    elif date_window == '60h':
        filter_date = dt.datetime.now()-dt.timedelta(hours=48)   
#     print(filter_date)
    df_filtered = df_source[(df_source['type'].isin(filter_type))& (df_source['score'] >= score_min) & (df_source['created_utc'] >= filter_date)]
    
    
    df_agg_ticker = df_filtered[['ticker','sentiment']].groupby(['ticker']).count().reset_index()
    df_agg_ticker.rename(columns={'sentiment': 'mentions'}, inplace = True)

    df_agg_ticker_sentiment = df_filtered.groupby(['ticker','sentiment'],as_index=False)['id'].count()
    df_agg_ticker_sentiment.rename(columns={'id': 'mentions'}, inplace = True)
    
    df_agg_ticker_sentiment = df_agg_ticker_sentiment.pivot(index="ticker", columns="sentiment", values="mentions")
    cols = df_agg_ticker_sentiment.columns.union(['negative', 'positive', 'neutral'], sort=False)
    df_agg_ticker_sentiment = df_agg_ticker_sentiment.reindex(cols, axis=1, fill_value=0)
    df_agg_ticker_sentiment = df_agg_ticker_sentiment.replace(np.nan, 0)
    df_agg_ticker_sentiment.reset_index(inplace = True)
    
    df_agg_ticker_sentiment = df_agg_ticker_sentiment.merge(df_agg_ticker, on=['ticker'])

    df_agg_ticker_sentiment.sort_values(by = ['mentions'], ascending = False, inplace = True)

    df_agg_ticker_sentiment_top_N = df_agg_ticker_sentiment.head(top_records)
    
    return df_agg_ticker_sentiment_top_N

In [5]:
df_agg_ticker_sentiment_top_N = get_agg_ticker_sentiment_top_N(flatten_data, submission_type = 'comment', score_min = 0,  date_window = 'all', top_records = 10)
tickers_top_N = list(df_agg_ticker_sentiment_top_N.ticker)
tickers_top_N

['NVDA', 'TSLA', 'LCID', 'AMD', 'ZM', 'PYPL', 'GME', 'AAPL', 'PLTR', 'BABA']

In [6]:
data = df_agg_ticker_sentiment_top_N[['ticker', 'positive', 'negative', 'neutral']].to_dict("list")
data

{'ticker': ['NVDA',
  'TSLA',
  'LCID',
  'AMD',
  'ZM',
  'PYPL',
  'GME',
  'AAPL',
  'PLTR',
  'BABA'],
 'positive': [86.0, 75.0, 40.0, 61.0, 31.0, 34.0, 39.0, 42.0, 38.0, 30.0],
 'negative': [42.0, 41.0, 22.0, 16.0, 24.0, 27.0, 22.0, 14.0, 22.0, 17.0],
 'neutral': [149.0, 103.0, 82.0, 62.0, 60.0, 53.0, 53.0, 52.0, 42.0, 40.0]}

# Load Prices from Yfinance

In [7]:
kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = 0,
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)

# Get data price


end = date.today()
start = end - pd.DateOffset(months=12)
Symbols = tickers_top_N#(['TSLA', 'NVDA'])
# create empty dataframe
stock_final = pd.DataFrame()
# iterate over each symbol
for i in Symbols:  
    
    # print the symbol which is being downloaded
    print( str(Symbols.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        state_means, _ = kf.filter(stock['Adj Close'])
    
        stock['Kalman Filter'] = state_means
        stock['signal'] = 0.0
        stock['signal'][:] = np.where(stock['Kalman Filter'][:] < stock['Adj Close'][:], 1.0, 0.0)

        stock['positions'] = stock['signal'].diff()
     
        stock['Middle line'] = "{:.0%}".format(0.5)
       
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None

        

prices = stock_final.pivot( columns = 'Name', values = ['Adj Close', 'Kalman Filter', 'Volume', 'signal', 'positions', 'Middle line'])



0 : NVDA,1 : TSLA,2 : LCID,

C:\Users\NLISIC~1\AppData\Local\Temp/ipykernel_3428/2435627139.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock['signal'][:] = np.where(stock['Kalman Filter'][:] < stock['Adj Close'][:], 1.0, 0.0)


3 : AMD,4 : ZM,5 : PYPL,6 : GME,7 : AAPL,8 : PLTR,9 : BABA,

In [8]:
prices.head()

Adj Close                                                      \
Name              AAPL        AMD        BABA        GME  LCID        NVDA   
Date                                                                         
2020-12-03  122.186745  92.309998  266.910004  16.120001  9.87  133.843109   
2020-12-04  121.500969  94.040001      267.25       16.9  9.88  135.464188   
2020-12-07  122.991791      94.07       264.0      16.35  9.94  135.948761   
2020-12-08  123.617912  92.919998  266.089996  16.940001  9.86  133.383514   
2020-12-09  121.033844  89.830002  263.799988      13.66  9.92  129.194672   

                                                           ... Middle line  \
Name             PLTR        PYPL        TSLA          ZM  ...        AAPL   
Date                                                       ...               
2020-12-03  24.030001  214.539993  593.380005  413.540009  ...         50%   
2020-12-04      23.85  217.770004  599.039978   410.01001  ...         50%   
2020-12-07  28.940001  217.929993   641.76001  407.850006  ...         50%   
2020-12-08      28.59  216.940002  649.880005       412.0  ...         50%   
2020-12-09  26.629999  210.800003   604.47998  385.359985  ...         50%   

                                                         
Name        AMD BABA  GME LCID NVDA PLTR PYPL TSLA   ZM  
Date                                                     
2020-12-03  50%  50%  50%  50%  50%  50%  50%  50%  50%  
2020-12-04  50%  50%  50%  50%  50%  50%  50%  50%  50%  
2020-12-07  50%  50%  50%  50%  50%  50%  50%  50%  50%  
2020-12-08  50%  50%  50%  50%  50%  50%  50%  50%  50%  
2020-12-09  50%  50%  50%  50%  50%  50%  50%  50%  50%  

[5 rows x 60 columns]

In [9]:
flatten_data_name_grouped_final = pd.DataFrame()

for ema in tickers_top_N:
    flatten_data_name = flatten_data[flatten_data['ticker'] == f"{ema}"]
    flatten_data_name_grouped = flatten_data_name.groupby([flatten_data_name.created_utc.dt.strftime('%d %b %Y')]).agg(
    {'positive': 'mean', 'negative': 'mean', 'id': 'size'})
    updated_list = [datetime.strptime(x, '%d %b %Y').strftime('%Y%m%d') for x in flatten_data_name_grouped.index.values.tolist()]
    
    mask = (prices.index > min(updated_list)) #& (df['date'] <= '2000-6-10')
    flatten_data_name_price = prices.loc[mask]
    flatten_data_name_price = flatten_data_name_price['Adj Close'][f"{ema}"]

    flatten_data_name_price.index = pd.to_datetime(flatten_data_name_price.index, format = '%m/%d/%Y').strftime('%d %b %Y')

    flatten_data_name_grouped  = pd.concat([flatten_data_name_price, flatten_data_name_grouped], axis=1, join='inner')

    flatten_data_name_grouped['price'] = flatten_data_name_grouped[f"{ema}"]
    del flatten_data_name_grouped[f"{ema}"]

    flatten_data_name_grouped['sum'] = flatten_data_name_grouped['positive'] + flatten_data_name_grouped['negative']

        
    flatten_data_name_grouped ['% of Bullish'] = flatten_data_name_grouped['positive']/flatten_data_name_grouped['sum']
    flatten_data_name_grouped ['% of Bearish'] = 1 - flatten_data_name_grouped ['% of Bullish']
    flatten_data_name_grouped['% of Bullish'] = flatten_data_name_grouped['% of Bullish'].astype(float).map("{:.2%}".format)
    flatten_data_name_grouped['% of Bearish'] = flatten_data_name_grouped['% of Bearish'].astype(float).map("{:.2%}".format)
    flatten_data_name_grouped['Middle line'] = "{:.0%}".format(0.5)

    if len(flatten_data_name_grouped) == 0:
        None
    else:
        flatten_data_name_grouped['Name']=ema
        flatten_data_name_grouped_final = flatten_data_name_grouped_final.append(flatten_data_name_grouped,sort=False)

        
bull_bearish = flatten_data_name_grouped_final.pivot( columns = 'Name', values = ['price', '% of Bullish', '% of Bearish', 'id', 'Middle line'])
  



In [10]:
bull_bearish.index

Index(['01 Dec 2021', '02 Dec 2021', '08 Nov 2021', '09 Nov 2021',
       '10 Nov 2021', '11 Nov 2021', '12 Nov 2021', '15 Nov 2021',
       '16 Nov 2021', '17 Nov 2021', '18 Nov 2021', '19 Nov 2021',
       '22 Nov 2021', '23 Nov 2021', '24 Nov 2021', '26 Nov 2021',
       '29 Nov 2021', '30 Nov 2021'],
      dtype='object')

In [ ]:
external_stylesheets = ['https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/lux/bootstrap.min.css' , 
                        'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css']
navbar = dbc.Nav(className="navbar-nav mr-auto", children=[
    dbc.DropdownMenu(label="Links", nav=True)
])

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = dbc.Container(fluid=True, children=[dbc.Row([
        dbc.Col([html.H2(children="Reddit / Wallstreetbet Sentiments Trading Dashboard",
                         style={'textAlign': 'centre',
                                'color': 'white'})], md=10),
        dbc.Col(md=2, children=[navbar])
    ], className='navbar navbar-expand-lg navbar-dark bg-primary'),
    html.Br(), html.Br(),
    dbc.Row([
        dbc.Col(xs=12, sm=12, md=12, lg=3, xl=3, children=[
            dbc.Col(html.H3(children='Select Ticker'),
                    style={'textAlign': 'left', 'font-weight': 'bold'}),
            dbc.Col(dcc.Dropdown(
                id='slider',
                options=[{'label': f'{i}-Ticker', 'value': i} for i in tickers_top_N],#[5, 6, 7, 8, 9, 10, 15, 20]],
                value=5)),
            html.Br(),
            dbc.Col(html.Div(id="output-panel")),
            html.Br(), html.Br(),
            dbc.Col(dbc.Card(body=True, className="card bg-light mb-3", children=[
                html.Div("About This Dashboard", className="card-header"),
                html.Div(className="card-body", children=[
                    html.P(children=["""This experiment aims to mine sentiments 
                    on Reddit/Wallstreetbet and derive trading signals based on their pre-market sentiments. 
                    Reddit/Wallstreetbet is a social media platform for retail traders to share their 
                    speculations and sentiments regarding any stock.""",
                                     html.Br(),
                                     html.Br(),
                                     """You can tweak the Ticker from the slider bar above to view the different sentiment
                                      performances.""",
                                     ], className="card-text")
                ])
            ])
                    )
        ]),

        dbc.Col(lg=9, xl=9, children=[
            dbc.Col(html.H3("WallStreetBets Ticker Sentiment"), width={"size": 7, "offset": 3},
                    style={'textAlign': 'center'}),
            dbc.Tabs(className="nav nav-pills", id='yaxis-column',
        children=[dbc.Tab(label=tickers_top_N[0], tab_id=tickers_top_N[0]),
          dbc.Tab(label=tickers_top_N[1], tab_id=tickers_top_N[1]), 
          dbc.Tab(label=tickers_top_N[2], tab_id=tickers_top_N[2]), 
          dbc.Tab(label=tickers_top_N[3], tab_id=tickers_top_N[3]), 
          dbc.Tab(label=tickers_top_N[4], tab_id=tickers_top_N[4]),
         dbc.Tab(label=tickers_top_N[5], tab_id=tickers_top_N[5]),
          dbc.Tab(label=tickers_top_N[6], tab_id=tickers_top_N[6]), 
          dbc.Tab(label=tickers_top_N[7], tab_id=tickers_top_N[7]), 
          dbc.Tab(label=tickers_top_N[8], tab_id=tickers_top_N[8]), 
          dbc.Tab(label=tickers_top_N[9], tab_id=tickers_top_N[9])]
                     #[
             
                     #]
                     ,
                     active_tab="$TSLA"),
            dcc.Graph(id='buy-sell-chart'),
            dcc.Graph(id='bull-bear-chart')
        ])
    ])
])


# Function for bull/bear graph
@app.callback(
    Output('bull-bear-chart', 'figure'),
    #Input('yaxis-column', 'active_tab'))
    Input('slider', 'value'))  
def update_graph_2(ema):
 

    fig = make_subplots(specs=[[{'secondary_y': True}]])

        #  Bullish Area 
    fig.add_trace(go.Scatter(
            x=bull_bearish.index, y=bull_bearish['% of Bullish'][f"{ema}"], #prices['% of Bullish']['TSLA'],
            mode='lines',
            name="Bullish",
            line=dict(width=1, color='rgba(0,102,0,0.3)'),
            stackgroup='one',
            groupnorm='percent'), secondary_y=False)

        #  Bearish Area 
    fig.add_trace(go.Scatter(
            x=bull_bearish.index, y=bull_bearish['% of Bearish'][f"{ema}"], #prices['% of Bearish']['TSLA'],
            mode='lines',
            name="Bearish",
            line=dict(width=1, color='rgba(190,23,23,0.3)'),
            stackgroup='one', fill='tonexty'), secondary_y=False)

        #  Chart
    fig.add_trace(go.Scatter(
            x=bull_bearish.index, y=bull_bearish['price'][f"{ema}"], #prices['Adj Close']['TSLA'],
            mode='lines',
            name=f"{ema} Closing Price",
            line=dict(width=2, color='black', dash='solid')),
            secondary_y=True)

        # 50% median line (Always visible)
    fig.add_trace(go.Scatter(
            x=bull_bearish.index, y=bull_bearish['Middle line'][f"{ema}"],
            mode='lines',
            name="50%",
            line=dict(width=0.5, color='black', dash='dot'),
            showlegend=False,
            hoverinfo='x'), secondary_y=False)

        # Add the top few buttons
    fig.update_xaxes(
            rangeslider_visible=False,
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(step="all")
                ])
            )
        )

        # Configure x & y axis & hovermode
    fig.update_layout(
            margin=dict(pad=4.5),
            title=dict(
                text=f"<b>Pre-Market Sentiments vs {ema} Performance</b>",
            ),
            template="plotly_white",
            hovermode="x",
            showlegend=True,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
            xaxis=dict(
                showgrid=True,
                showline=False,
                tickmode='auto',
                nticks=7,
                fixedrange=True,
                range=[min(bull_bearish.index), max(bull_bearish.index)]),
            yaxis=dict(
                # automargin=True,
                type='linear',
                range=[0, 100],
                showgrid=False,
                ticksuffix='%',
                fixedrange=True,
                tickvals=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]),
            # Converted into the dropdown menu option
            yaxis2=dict(
                # automargin=True,
                type='linear',
                showgrid=False,
                fixedrange=True,
                ticksuffix=' USD')
            #         range=[0, 200])
        )

        # Configure spikes
    fig.update_xaxes(showspikes=True, spikecolor="black", spikesnap="cursor", spikedash='dot', spikemode="across",
                         tickformat='%d %b %y', spikethickness=1)
    fig.update_yaxes(title_text="<b>% of Bulls vs Bears</b>", showspikes=False, spikecolor="grey",
                         spikethickness=0.25)
    fig.update_yaxes(title_text="<b>Closing Price</b>", secondary_y=True)

    return fig


# Function to render Portfolio Chart
@app.callback(
    Output('buy-sell-chart', 'figure'),

    Input('slider', 'value'))
def update_graph(ema):

        
    data = [
            go.Scatter(x=prices.index, y=(prices['Adj Close'][f"{ema}"]),
                       mode='lines', name=f"{ema} Closing Price",
                       line=dict(color='#86d3e3', width=2)),
            go.Scatter(x=prices.index, y=(prices['Kalman Filter'][f"{ema}"]),
                       mode='lines', name=f"{ema} Kalman Filter Price",
                       line=dict(color='#e39686', width=2)),
            go.Scatter(x=prices.loc[prices['positions'][f"{ema}"] == 1.0].index,
                       y=(prices['Adj Close'][f"{ema}"][prices['positions'][f"{ema}"] == 1.0]),
                       mode='markers', name="Buy",
                       marker=dict(symbol='triangle-up', color="green", size=7)),
            go.Scatter(x=prices.loc[prices['positions'][f"{ema}"] == -1.0].index,
                       y=(prices['Adj Close'][f"{ema}"][prices['positions'][f"{ema}"] == -1.0]),
                       mode='markers', name="Sell",
                       marker=dict(symbol='triangle-down', color="red", size=7))
        
        ]
        
        
    fig = go.Figure(data=data)
    fig.update_xaxes(
            rangeslider_visible=False,
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                 
                    dict(step="all")
                ])
            )
        )

    fig.update_layout(
            title=f"<b>Long/ Short Signal {ema}</b>",
            template="plotly_white",
            hovermode="x",
            showlegend=True,
            legend=dict(
                x=0.4,
                y=1.15,
                orientation='h'),
            xaxis=dict(
                showgrid=True,
                showline=True,
                tickmode='auto',
                fixedrange=True,
                range=[min(prices.index), max(prices.index)]),
            yaxis=dict(
                type='linear',
                showline=False,
                showgrid=False,
                fixedrange=True,
                ticksuffix=' USD'
            ))
    fig.update_xaxes(showspikes=True, spikecolor="red", spikesnap="cursor", spikemode="across",
                         tickformat='%d %b %y')
    fig.update_yaxes(showspikes=True, spikecolor="grey", spikethickness=2)
    return fig
        
        
        
   


# this is needed for the procfile to deploy to heroku
server = app.server   
if __name__ == '__main__':
    app.run_server()
    
    
    
    


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
